思路是：先用 Alpha158 handler 取出特征矩阵，然后基于 statsmodels 做 ADF/KPSS 平稳性检验，和基于相关系数矩阵 + VIF 做共线性诊断。

## 1) 取出 Alpha158 因子数据（Panel → MultiIndex DataFrame）

In [1]:
import qlib
from qlib.data import D
from qlib.contrib.data.handler import Alpha158
from qlib.workflow import R
import pandas as pd
qlib.init(provider_uri="~/.qlib/qlib_data/cn_data")  # 根据你的路径调整

# 定义股票池与时间窗
universe = D.instruments(market="csi300")     # 例：沪深300
start, end = "2016-01-01", "2022-12-31"

# 使用 Alpha158 handler 计算特征
handler = Alpha158(instruments=universe, start_time=start, end_time=end, fit_start_time=start, fit_end_time=end)
df_feat = handler.fetch(col_set="feature") 

# 可选：与label对齐或去极值/标准化，按你后续建模流程来
# 这里先简单清洗
df_feat = df_feat.sort_index().dropna(how="all")  # 行全空去掉


[74066:MainThread](2025-09-02 15:59:45,992) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[74066:MainThread](2025-09-02 15:59:45,993) INFO - qlib.Initialization - [__init__.py:79] - qlib successfully initialized based on client settings.
[74066:MainThread](2025-09-02 15:59:45,993) INFO - qlib.Initialization - [__init__.py:81] - data_path={'__DEFAULT_FREQ': PosixPath('/Users/huhao/.qlib/qlib_data/cn_data')}
[74066:MainThread](2025-09-02 16:00:03,027) INFO - qlib.timer - [log.py:127] - Time cost: 17.032s | Loading data Done
[74066:MainThread](2025-09-02 16:00:03,157) INFO - qlib.timer - [log.py:127] - Time cost: 0.053s | DropnaLabel Done
[74066:MainThread](2025-09-02 16:00:03,573) INFO - qlib.timer - [log.py:127] - Time cost: 0.415s | CSZScoreNorm Done
[74066:MainThread](2025-09-02 16:00:03,576) INFO - qlib.timer - [log.py:127] - Time cost: 0.549s | fit & process data Done
[74066:MainThread](2025-09-02 16:00:03,576) INFO - qlib.timer - [log.py:127] - Time cost: 17.

In [3]:
df_feat[['KMID','KLEN','KMID2','KUP','KUP2']]

KMID      KLEN     KMID2       KUP      KUP2
datetime   instrument                                                  
2016-01-04 SH600000   -0.026258  0.039934 -0.657535  0.000000  0.000000
           SH600008   -0.099804  0.105675 -0.944445  0.003914  0.037037
           SH600009   -0.042668  0.046393 -0.919709  0.000339  0.007299
           SH600010   -0.099448  0.104972 -0.947369  0.002762  0.026316
           SH600011   -0.055046  0.060780 -0.905662  0.003440  0.056603
...                         ...       ...       ...       ...       ...
2022-12-30 SZ300896   -0.003063  0.023940 -0.127942  0.004418  0.184562
           SZ300979   -0.007301  0.023814 -0.306569  0.008170  0.343066
           SZ300999    0.000919  0.017004  0.054054  0.006664  0.391889
           SZ301236   -0.011798  0.033146 -0.355932  0.016292  0.491525
           SZ301269    0.000778  0.013667  0.056908  0.006667  0.487806

[513680 rows x 5 columns]

## 2) 平稳性检测（ADF + KPSS）

要点

- 因子是时间序列，做 时间向 的单位根检验。
- 按（股票、因子）分别做 ADF；结果在横截面上做聚合（如通过率、p 值分布）。
- ADF 原假设“非平稳”；KPSS 原假设“平稳”。两者结合更稳健。

In [ ]:
# 2. 安全的 ADF/KPSS wrapper
def safe_adf(series, min_n=50, regression='c', autolag='AIC'):
    out = {'adf_stat': np.nan, 'adf_p': np.nan, 'adf_note': None, 'adf_error': None}
    ts = series.dropna()
    if len(ts) < min_n or ts.nunique() <= 1 or ts.std() < 1e-12:
        out['adf_error'] = 'short_or_constant'
        return out
    try:
        with warnings.catch_warnings(record=True) as w:
            warnings.simplefilter("always")
            stat, p, *_ = adfuller(ts.values, autolag=autolag, regression=regression)
            if any(isinstance(x.message, InterpolationWarning) for x in w):
                out['adf_note'] = 'interp_warning'
                p = np.nan
            out.update({'adf_stat': stat, 'adf_p': p})
            return out
    except Exception as e:
        out['adf_error'] = str(e)
        return out

def safe_kpss(series, min_n=50, regression='c', nlags='auto'):
    out = {'kpss_stat': np.nan, 'kpss_p': np.nan, 'kpss_note': None, 'kpss_error': None}
    ts = series.dropna()
    if len(ts) < min_n or ts.nunique() <= 1 or ts.std() < 1e-12:
        out['kpss_error'] = 'short_or_constant'
        return out
    try:
        with warnings.catch_warnings(record=True) as w:
            warnings.simplefilter("always")
            stat, p, *_ = kpss(ts.values, regression=regression, nlags=nlags)
            if any(isinstance(x.message, InterpolationWarning) for x in w):
                out['kpss_note'] = 'interp_warning'
                p = np.nan
            out.update({'kpss_stat': stat, 'kpss_p': p})
            return out
    except Exception as e:
        out['kpss_error'] = str(e)
        return out

# 3. Panel 检测逻辑
def panel_adf_kpss(df_feat, min_n=80):
    rows = []
    for fac in df_feat.columns:
        mat = df_feat[fac].unstack(level=1)
        for inst in mat.columns:
            s = mat[inst]
            adf_r = safe_adf(s, min_n=min_n)
            kpss_r = safe_kpss(s, min_n=min_n)
            row = {'factor': fac, 'instrument': inst, 'n_obs': s.dropna().shape[0]}
            row.update(adf_r)
            row.update(kpss_r)
            rows.append(row)
    return pd.DataFrame(rows)

In [ ]:
res_df = panel_adf_kpss(df_feat, min_n=80)

# 4. 聚合报表（修复后的写法）
summary = res_df.groupby("factor").agg({
    "n_obs": "count",
    "adf_p": lambda x: np.mean(x < 0.05),
    "kpss_p": lambda x: np.mean(x > 0.05),
})

both_pass = res_df.groupby("factor").apply(
    lambda g: np.mean((g["adf_p"] < 0.05) & (g["kpss_p"] > 0.05))
)
summary["both_pass_rate"] = both_pass

print("==== Top 10 最稳定因子 ====")
print(summary.sort_values("both_pass_rate", ascending=False).head(10))
print("\n==== Bottom 10 最不稳定因子 ====")
print(summary.sort_values("both_pass_rate", ascending=True).head(10))



## Top 10 最稳定因子

这些因子（如 `WVMA30`, `CORR10`, `STD5` 等）在 **大部分股票上都能通过 ADF 和 KPSS 的平稳性检验**：

* **`adf_p` 列**：表示 ADF p 值 < 0.05 的比例

  * 比如 `CORR10` 的 0.956790 ≈ 95.7% 股票在 ADF 下判定为平稳。

* **`kpss_p` 列**：表示 KPSS p 值 > 0.05 的比例

  * `CORR10` 的 0.148148 ≈ 14.8% 股票在 KPSS 下判定为平稳。

* **`both_pass_rate` 列**：两个检验同时通过的比例

  * `WVMA30` 大约 14.6%，在所有因子里算是比较高的。

👉 结论：这些因子总体上 **比较平稳**，但也不是完美，大部分因子真正双检验都过的比例只有 10–15%，说明 **股票层面的因子时间序列大多还是弱平稳或存在单位根**。

---

## Bottom 10 最不稳定因子

典型代表：`RESI20`, `RESI10`, `RESI30`, `RESI60`，以及一系列 `VSUM*` 系列。

* **RESI 系列**：

  * `adf_p` 非常高（>0.9），说明几乎所有股票在 ADF 下都被判为「非平稳」。
  * `kpss_p` 接近 0，说明在 KPSS 下也被判为「非平稳」。
  * `both_pass_rate` = 0，彻底不平稳。

* **VWAP0**：ADF 和 KPSS 两个检验都挂零，说明它的序列几乎完全没有平稳性。

* **VSUMP/VSUMN/VSUMD 系列**：`kpss_p` 在 0.8–1.8% 之间，也基本不平稳。

👉 结论：
这些因子序列要么趋势很强、要么波动模式不稳定，**对时间序列模型（如 IC 回归、时序预测）非常不友好**。在建模时需要：

* 考虑 **差分（diff）或标准化** 来处理。
* 或者直接 **剔除掉这些因子**，避免引入噪声。

---

⚖️ **整体情况**

* 在 Alpha158 里，确实只有一小部分因子在股票横截面上表现出较好的平稳性。
* 像 `RESI*` 这种残差相关的指标，大概率是 **非平稳的“价格驱动型因子”**。
* 反而一些基于波动率（`STD*`）、移动平均（`WVMA*`）、相关性（`CORR*`）的因子，平稳性更好。

In [ ]:
# 5. 可视化某个因子的分布
fac = summary.sort_values("both_pass_rate").index[-1]
sub = res_df[res_df['factor'] == fac]
plt.figure(figsize=(8,4))
plt.scatter(sub['adf_p'], sub['kpss_p'], alpha=0.6)
plt.axvline(0.05, color='red', ls='--')
plt.axhline(0.05, color='red', ls='--')
plt.xlabel("ADF p-value (<0.05=stationary)")
plt.ylabel("KPSS p-value (>0.05=stationary)")
plt.title(f"{fac} 平稳性检测 (股票横截面)")
plt.show()

In [ ]:
这是针对 **因子 WVMA30 ** 做的 **平稳性检测结果（横截面股票层面）**：

---

### 图表内容

* **横轴：ADF p-value**

  * 判别标准：`< 0.05` → 认为序列平稳。
  * 图中红色虚线在 `0.05` 处。

* **纵轴：KPSS p-value**

  * 判别标准：`> 0.05` → 认为序列平稳。
  * 图中红色虚线在 `0.05` 处。

* **散点：每个点代表某只股票在该因子上的序列检验结果**

  * 越靠左（ADF p 越小）说明平稳性越强。
  * 越靠上（KPSS p 越大）说明平稳性越强。
  * **理想的因子**应该分布在左上象限（ADF < 0.05 且 KPSS > 0.05）。


常见处理

- 对“明显非平稳”的因子：做差分 / 对数差分 / 滚动去趋势（如 s - s.rolling(60).mean()）后重检。
- 也可只在训练集时间窗内做检验，避免数据泄露。

### 3) 多重共线性检测（相关矩阵 + VIF）

要点

- 共线性是横截面问题：同一交易日、不同因子之间的线性相关。
- 做法：选取一个代表性日期（或在多日上分别统计），对因子矩阵 X_t 计算相关系数矩阵&VIF。
- 阈值经验：|ρ|>0.9 视为高度相关；VIF>10 视为严重共线（可调为 5）。

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 取一个日期的横截面（也可对多日循环后求均值/分位数）
a_date = df_feat.index.get_level_values(0).unique()[-1]   # 取样例的最后一天
X = df_feat.xs(a_date, level=0)  # index=instrument, columns=factors
X = X.dropna(axis=1, thresh=int(0.8*len(X)))  # 丢弃缺失过多的因子
X = X.dropna(axis=0)                           # 丢掉有缺失的股票
# 简单标准化（便于数值稳定）
X = (X - X.mean())/X.std(ddof=0)

# 1) 相关性矩阵
corr = X.corr().abs()

# 2) VIF（逐列回归的 1/(1-R^2)）
vif = pd.Series(
    [variance_inflation_factor(X.values, i) for i in range(X.shape[1])],
    index=X.columns, name="VIF"
).sort_values(ascending=False)

# 3) 基于阈值的自动剔除（示例：先剔高度相关，再按VIF去冗余）
#   先相关性去冗余：保留每对高度相关(|ρ|>0.95)中的重要性更高者（这里用全样本方差占位，你也可以换成IC/重要性等）
to_drop = set()
corr_thr = 0.95
fac_var = df_feat.var()  # 全样本上的因子方差，示意用途
for i in corr.columns:
    if i in to_drop: 
        continue
    for j in corr.index:
        if i == j or j in to_drop: 
            continue
        if corr.loc[j, i] > corr_thr:
            # 保留方差更大的（更“有信息量”的）因子
            drop = j if fac_var[j] < fac_var[i] else i
            to_drop.add(drop)

X_red = X.drop(columns=list(to_drop))

# 再按 VIF 阈值迭代剔除
def drop_by_vif(X, thr=10):
    while True:
        vif_s = pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns)
        worst = vif_s.max()
        if worst <= thr:
            return X, vif_s.sort_values(ascending=False)
        X = X.drop(columns=[vif_s.idxmax()])

X_final, vif_final = drop_by_vif(X_red, thr=10)

# 结果：
# - corr：因子间相关性矩阵
# - vif：初始VIF排序
# - X_final/vif_final：去冗余后的因子子集与其VIF


In [ ]:
corr

In [ ]:
vif

In [ ]:
X_final

In [ ]:
vif_final

实务建议

- 不要只看某一天：可在多日上重复计算，将“高相关对出现频次”与“VIF 均值/分位数”作为依据，挑出长期共线的因子对再做剔除。
- 如果你后续用树模型（如 LGBM），对共线性不敏感；但在线性/稀疏模型里（如 Lasso/线回归）就很重要。
- 也可用 PCA/正交化（如对相关簇做 PCA 取前主成分）作为“去共线”的替代方案。

## 4) 把两件事串成流程（示例）

训练集期间做 ADF+KPSS，过滤掉“通过率过低”的因子（例如 both_pass_rate < 0.6）。

对剩余因子，在多日横截面上统计相关矩阵和 VIF，按阈值与业务偏好（可结合 IC、特征重要性）剔除冗余。

记录保留名单，固定到后续训练/回测配置中，避免数据泄露。